In [23]:
import requests
import json
import numpy as np

query_string = '''
{
    moloches(first: 1000, where: {id_gt: lastID}) {
      id
      version
      summoner
      newContract
      deleted
      summoningTime
      createdAt
      periodDuration
      votingPeriodLength
      gracePeriodLength
      proposalDeposit
      dilutionBound
      processingReward
      guildBankAddress
      guildBankBalanceV1
      members {
        id
        createdAt
        kicked
        jailed
        didRagequit
        shares
        loot
        votes
      }
      totalShares
      totalLoot
    }
  }
  '''

def run_all_moloch_query(q, endpoint="mainnet"):

    # The Graph DAOhaus Mainnet endpoint
    # https://api.thegraph.com/subgraphs/name/odyssy-automaton/daohaus
    # The Graph DAOhaus xDAI endpoint
    # https://api.thegraph.com/subgraphs/name/odyssy-automaton/daohaus-xdai
    
    assert endpoint in ["mainnet", "xdai"]
    if endpoint == "mainnet":
        QUERY_URL = "https://api.thegraph.com/subgraphs/name/odyssy-automaton/daohaus"
    elif endpoint == "xdai":
        QUERY_URL = "https://api.thegraph.com/subgraphs/name/odyssy-automaton/daohaus-xdai"

    all_moloches = []
    new_str = q.replace("lastID", '""')
    
    response = requests.post(
       QUERY_URL, "", json={"query": new_str}
    )
    while json.loads(response.text)["data"]["moloches"]:
        if response.status_code == 200:
            all_moloches.extend(json.loads(response.text)["data"]["moloches"])
        else:
            raise Exception("Query failed. Return code is {}.   {}".format(response.status_code, q))
        new_str = q.replace("lastID", '"' + all_moloches[-1]["id"] + '"')
        response = requests.post(
            QUERY_URL, "", json={"query": new_str}
        )
        
    print(f"Total {endpoint} Moloches: {len(all_moloches)}")
    return all_moloches
    
all_mainnet_moloches = run_all_moloch_query(query_string, "mainnet")
all_xdai_moloches = run_all_moloch_query(query_string, "xdai")
all_moloches = all_mainnet_moloches + all_xdai_moloches
print(len(all_moloches))

Total Moloches: 367
Total Moloches: 873
1240


In [24]:
def get_member_metrics(dao_info):

    kicked_member_count = 0
    rage_quit_member_count = 0
    members_in_multiple_daos = 0
    total_dao_count = 0
    current_member_count = 0
    member_shares_dict = {}

    member_metrics_dict = {}

    for member in dao_info["members"]:
        memberAddress = member["id"].split("-")[2]
        if member["kicked"]:
            kicked_member_count += 1
            continue
        if member["didRagequit"]:
            rage_quit_member_count += 1
            continue
        else:
            current_member_count += 1
            member_shares_dict[memberAddress] = member['shares']


    # metrics (keys) that are being generated
    member_metrics_dict["all_time_member_count"] = len(dao_info["members"])
    member_metrics_dict["current_member_count"] = current_member_count
    member_metrics_dict["kicked_member_count"] = kicked_member_count
    member_metrics_dict["rage_quit_member_count"] = rage_quit_member_count

    return member_metrics_dict, member_shares_dict


all_shares = []
all_metrics = []
for dao in all_moloches:
    current_member_metrics, current_shares_dict = get_member_metrics(dao)
    if len(current_shares_dict.keys()) > 1:
        all_metrics.append(current_member_metrics)
        all_shares.append(current_shares_dict)

In [27]:
import numpy as np

minimum_coalition_list = []
for share_dict in all_shares:
    
    share_list = [int(x) for x in share_dict.values()]
    share_list.sort(reverse=True)
    total_sum = sum(share_list)
    running_total = 0
    min_coalition = len(share_list)
    for i in range(len(share_list)):
        running_total += share_list[i]
        if running_total > (total_sum/2):
            min_coalition = i + 1
            break
    minimum_coalition_list.append(min_coalition/len(share_list))
    
minimum_coalition_list = np.array(minimum_coalition_list)
twenty_five = np.percentile(minimum_coalition_list, 25)
fifty = np.percentile(minimum_coalition_list, 50)
seventy_five = np.percentile(minimum_coalition_list, 75)
print(f"25%: {twenty_five} \n 50%: {fifty} \n 75%: {seventy_five}")

25%: 0.3333333333333333 
 50%: 0.5 
 75%: 0.6666666666666666
